<a href="https://colab.research.google.com/github/Myeong2/ComputerVision_Project3-DrinkDetector/blob/master/%EC%95%A1%ED%84%B0_%ED%81%AC%EB%A6%AC%ED%8B%B1_%EC%82%AC%EC%9A%A9_%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

lr = 0.0002
gamma = 0.98

def main():
    env = gym.make('CartPole-v1')
    model = ActorCritic(lr) # 학습률(lr)을 인자로 넘겨줍니다
    n_rollout = 5 # 5번의 step마다 update 진행
    print_int = 20
    score = 0

    for episode in range(2000):
        done = False
        state = env.reset()
        while not done:
            for t in range(n_rollout):
                prob = model.pi(torch.from_numpy(state).float()) # 확률로 변환
                actions = Categorical(prob).sample() # 확률에 따라 행동을 샘플링
                state_, returns, done, info = env.step(actions.item())
                model.put_data((state, actions, returns, state_, done))
                state = state_ # 다음 상태로 넘어감
                score += returns # 리턴을 누적해서 더함

                if done:
                    break

            model.train() # 모델 학습을 위해 train 함수를 호출

        if episode % print_int == 0 and episode != 0:
            print('에피소드: {}, 평균 점수: {}'.format(episode, score / print_int))
            score = 0

    env.close()

class ActorCritic(nn.Module):
    def __init__(self, lr):
        super(ActorCritic, self).__init__()
        self.data = []
        self.fc_common = nn.Linear(4, 128)
        self.fc_pi = nn.Linear(128, 2)
        self.fc_v = nn.Linear(128, 1)
        self.opt = optim.Adam(self.parameters(), lr=lr)

    ## REINFORCE와는 다르게, 훈련시켜야하는 network가 2개(pi & v)다
    def pi(self, x, dim=0): # 수정: "sef" 대신 "self" 사용, dim의 기본값을 0으로 설정하여 배치 차원에 대해 softmax 적용
        x = F.relu(self.fc_common(x))
        x = self.fc_pi(x)
        pi = F.softmax(x, dim=dim) # 각 행동에 대한 확률 반환
        return pi

    def v(self, x):
        x = F.relu(self.fc_common(x))
        v = self.fc_v(x)
        return v

    def put_data(self, item):
        self.data.append(item)

    def batch(self):
        S, A, R, S_, Done = [], [], [], [], []

        for item in self.data:
            s, a, r, s_, done = item
            S.append(s)
            A.append([a])
            R.append([r / 100.0])
            S_.append(s_)
            if done:
                d = 0
            else:
                d = 1
            Done.append([d])

        s_batch = torch.tensor(S, dtype=torch.float)
        a_batch = torch.tensor(A, dtype=torch.float) # 수정: 쉼표(,) 제거
        r_batch = torch.tensor(R, dtype=torch.float) # 수정: 쉼표(,) 제거
        s2_batch = torch.tensor(S_, dtype=torch.float) # 수정: 쉼표(,) 제거
        d_batch = torch.tensor(Done, dtype=torch.float) # 수정: 쉼표(,) 제거
        self.data = []

        return s_batch, a_batch, r_batch, s2_batch, d_batch

    def train(self):
        s, a, r, s_, done = self.batch()
        with torch.no_grad():
            v_s_ = self.v(s_).squeeze()  # 불필요한 차원을 제거하여 1차원 텐서로 변환
            TD_target = r + gamma * v_s_ * done  # TD 타겟 계산
            TD_error = TD_target - self.v(s).squeeze()  # 불필요한 차원을 제거하여 1차원 텐서로 변환

        pi = self.pi(s, dim=1)
        pi_a = pi.gather(1, a.long())
        loss = -torch.log(pi_a) * TD_error.detach() + F.smooth_l1_loss(self.v(s), TD_target.detach())

        self.opt.zero_grad()
        loss.mean().backward()
        self.opt.step()

if __name__ == "__main__":
    main()


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
<ipython-input-1-d56456db05b1>:80: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  s_batch = torch.tensor(S, dtype=torch.float)
<ipython-input-1-d56456db05b1>:98: UserWarning: Using a ta

에피소드: 20, 평균 점수: 20.75
에피소드: 40, 평균 점수: 15.8
에피소드: 60, 평균 점수: 16.9
에피소드: 80, 평균 점수: 16.8
에피소드: 100, 평균 점수: 19.75
에피소드: 120, 평균 점수: 17.25
에피소드: 140, 평균 점수: 17.2
에피소드: 160, 평균 점수: 16.05
에피소드: 180, 평균 점수: 17.7
에피소드: 200, 평균 점수: 16.1
에피소드: 220, 평균 점수: 16.3
에피소드: 240, 평균 점수: 19.35
에피소드: 260, 평균 점수: 18.8
에피소드: 280, 평균 점수: 18.7
에피소드: 300, 평균 점수: 18.3
에피소드: 320, 평균 점수: 25.4
에피소드: 340, 평균 점수: 20.65
에피소드: 360, 평균 점수: 22.85
에피소드: 380, 평균 점수: 20.4
에피소드: 400, 평균 점수: 25.45
에피소드: 420, 평균 점수: 18.9
에피소드: 440, 평균 점수: 21.8
에피소드: 460, 평균 점수: 21.3
에피소드: 480, 평균 점수: 25.85
에피소드: 500, 평균 점수: 29.4
에피소드: 520, 평균 점수: 32.55
에피소드: 540, 평균 점수: 31.5
에피소드: 560, 평균 점수: 27.2
에피소드: 580, 평균 점수: 31.95
에피소드: 600, 평균 점수: 32.7
에피소드: 620, 평균 점수: 35.95
에피소드: 640, 평균 점수: 33.2
에피소드: 660, 평균 점수: 41.05
에피소드: 680, 평균 점수: 32.65
에피소드: 700, 평균 점수: 40.95
에피소드: 720, 평균 점수: 41.25
에피소드: 740, 평균 점수: 43.95
에피소드: 760, 평균 점수: 28.95
에피소드: 780, 평균 점수: 39.65
에피소드: 800, 평균 점수: 50.1
에피소드: 820, 평균 점수: 46.1
에피소드: 840, 평균 점수: 39.65
에피소드: 860, 평균 점수: 